In [1]:
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
import joblib
from collections import Counter
import cv2

height, width = 64, 64

In [2]:
data = pd.read_csv(f"../archive/ascii_character_classification_{height}_x_{width}.csv", header=0).sample(frac=.25)
label_counts = Counter(data.iloc[:, 0])
print(label_counts)

Counter({0: 1231, 38: 150, 50: 145, 54: 144, 7: 138, 48: 137, 37: 137, 3: 136, 29: 135, 15: 134, 32: 133, 19: 133, 53: 133, 93: 132, 16: 131, 79: 131, 63: 131, 81: 130, 88: 130, 44: 128, 36: 128, 89: 128, 23: 128, 31: 128, 78: 128, 69: 127, 75: 127, 77: 126, 46: 126, 76: 126, 73: 126, 13: 125, 83: 125, 42: 125, 86: 125, 2: 125, 20: 125, 52: 124, 47: 123, 84: 123, 10: 123, 74: 122, 59: 122, 25: 121, 18: 121, 82: 120, 5: 120, 28: 119, 43: 119, 64: 119, 11: 119, 80: 119, 85: 119, 67: 118, 66: 117, 55: 116, 27: 115, 60: 115, 1: 115, 12: 115, 9: 115, 4: 114, 61: 114, 72: 113, 17: 113, 6: 113, 71: 112, 34: 111, 21: 111, 8: 111, 35: 110, 95: 110, 14: 110, 91: 109, 57: 109, 22: 109, 94: 108, 45: 108, 87: 108, 40: 108, 49: 107, 65: 107, 41: 105, 26: 103, 70: 103, 58: 101, 68: 101, 33: 100, 62: 100, 90: 100, 39: 99, 24: 98, 30: 97, 51: 94, 56: 90, 92: 88})


In [3]:
X = data.iloc[:, 1:].astype("float64")
y = data.iloc[:, 0].astype("float64")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# K-NN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
joblib.dump(knn, f'../artifacts/knn_model_{height}_x_{width}.pkl')

['../artifacts/knn_model_64_x_64.pkl']

In [5]:
y_pred = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = knn.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (2500, 4096)
Train Accuracy: 72.3800%
Test Accuracy: 48.3200%
F1 Score: 48.7384%
Recall: 48.3200%


In [6]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        image_reshaped = image.reshape((height, width))
        features = hog(image_reshaped, pixels_per_cell=(2, 2), cells_per_block=(1, 1), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_hog = extract_hog_features(np.array(X))
X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)

# %%
# K-NN Classifier with HOG features
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
joblib.dump(knn, f'../artifacts/knn_model_hog_{height}_x_{width}.pkl')

# %%
y_pred = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = knn.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (2500, 9216)
Train Accuracy: 55.3800%
Test Accuracy: 35.0400%
F1 Score: 34.9726%
Recall: 35.0400%


In [7]:
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    
    for image in images:
        image_reshaped = image.reshape((height, width)).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(image_reshaped, None)
        
        # If no keypoints are found, use a zero array of the same length as a typical descriptor
        if descriptors is None:
            descriptors = np.zeros((1, sift.descriptorSize()), dtype=np.float32)
        
        # Flatten descriptors and use them as features
        features = descriptors.flatten()
        sift_features.append(features)
    
    return np.array(sift_features)

# Extract SIFT features
X_sift = extract_sift_features(np.array(X))

# %%
X_sift = extract_sift_features(np.array(X))

# Since the number of features might vary, we need to ensure consistent feature vector size
# Here, we'll pad with zeros to the maximum descriptor length found
max_len = max(len(f) for f in X_sift)
X_sift = np.array([np.pad(f, (0, max_len - len(f)), 'constant') for f in X_sift])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sift, y, test_size=0.2, random_state=42)

# %%
# K-NN Classifier with SIFT features
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
joblib.dump(knn, f'../artifacts/knn_model_sift_{height}_x_{width}.pkl')

# %%
y_pred = knn.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = knn.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

test Shape: (2500, 128)
Train Accuracy: 1.0300%
Test Accuracy: 0.8800%
F1 Score: 0.0154%
Recall: 0.8800%


: 